# ADM1 Model Tutorial

This notebook demonstrates how to use the **ADM1Model** (Anaerobic Digestion Model No. 1) from OpenAD-lib for simulating biogas production from anaerobic digestion processes. 

It covers:
1. Generating influent data using the ACoD (Anaerobic Co-Digestion) method.
2. Initializing the ADM1 Model.
3. Running a dynamic simulation.
4. Visualizing the results.

## 1. Setup and Imports

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from openad_lib.models.mechanistic.adm1_model import ADM1Model
from openad_lib.preprocessing import acod

print("Imports successful!")

## 2. Generate Influent Data (ACoD)

The ADM1 model requires detailed characterization of the influent (feedstock). The ACoD module allows us to generate these state variables from substrate ratios.

In [ ]:
# Define paths
base_path = os.path.dirname(os.getcwd())
ratio_file = os.path.join(base_path, 'src', 'openad_lib', 'data', 'feedstock', 'Feed_Data.csv')
output_file = 'ADM1_Influent_Generated.csv'

if os.path.exists(ratio_file):
    print(f"Reading ratios from: {ratio_file}")
    influent_df = acod.generate_influent_data(ratio_file, output_file)
    print("Influent characterization complete.")
    print(f"Shape: {influent_df.shape}")
    display(influent_df.head())
else:
    print("Feed/Ratio data file not found!")

## 3. Initialize the ADM1 Model

We create an instance of the `ADM1Model`. 
This model currently uses the standard ADM1 kinetics and parameters.

In [ ]:
model = ADM1Model()
print("ADM1 Model Initialized")

## 4. Run Simulation

We use the `simulate` method, passing in our generated influent DataFrame. 
The simulation will run for the duration specified in the input file (rows).

In [ ]:
print("Starting simulation...")
simulation_output = model.simulate(influent_df)
print("Simulation complete.")

## 5. Analyze Results

The output dictionary contains:
- `results`: DataFrame with all 35+ state variables over time.
- `q_gas`: Total biogas flow rate.
- `q_ch4`: Methane flow rate.

In [ ]:
results = simulation_output['results']
q_gas_df = simulation_output['q_gas']
q_ch4_df = simulation_output['q_ch4']

results.head()

In [ ]:
plt.style.use('bmh')
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(q_gas_df['time'], q_gas_df['q_gas'], label='Total Biogas', linewidth=2)
ax.plot(q_ch4_df['time'], q_ch4_df['q_ch4'], label='Methane (CH4)', linestyle='--', linewidth=2)

ax.set_xlabel('Time (days)')
ax.set_ylabel('Flow Rate (m³/day)')
ax.set_title('Simulated Biogas Production')
ax.legend()
plt.show()

## 6. Inspect Specific State Variables

We can look at specific concentrations in the reactor, such as VFAs (Acetate, Propionate, Butyrate) or pH (if calculated/available).

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(results['time'], results['S_ac'], label='Acetate')
ax.plot(results['time'], results['S_pro'], label='Propionate')
ax.plot(results['time'], results['S_bu'], label='Butyrate')

ax.set_xlabel('Time (days)')
ax.set_ylabel('Concentration (kg COD/m³)')
ax.set_title('VFA Dynamics')
ax.legend()
plt.show()